# **We do everything from scratch with 2 model pipeline vision and 0.85 acc goal**

### **1. Split**

In [1]:
print("helloworld")

helloworld


### **2. Construct 2 model pipeline based on treshold**

### **3. Tune feature qty and which features work for each treshold range, it could be different qtys and features** 

### **4. Select Models and hiperparameter tuning**

Step 4 and 3 can be in inverse order or iterative

### **5. If we are still at less than 0.85 acc then we go back to parser and maybe tune features or get new ones**

### **6. If we still at less than 0.85 acc then we switch our approach and basically do regression and round up instead of classifying**